# Mesa overview

**agent based models** are compute simulations involving multiple entities **acting and interacting** with one another based on their programmed behavior. Agents can be used to represent living cells, animals, individual humand, even entire organizations or abstract entities.

Sometimes, we may have an understanding of how the individual components of a system behave, and want to see what system-level behaviors and effects emerge from their interaction. Other times, we may have a good idea of how the system overall behaves, and want to figure out what individual behaviors explain it. Or we may want to see how to get agents to cooperate or compete most effectively. Or we may just want to build a cool toy with colorful little dots moving around.

Meas modules containes three parts:

1. Modeling, build the model themselves with a **model** and **agent classes**, a **scheduler** to determine the sequences in which the agents act, and **sapce** for them to move around on

2. Analysis, collected data from your model, or to run it multiple times with differenet parameter values

3. Visualizations

ps: If the vscode show the "Running cells with 'sim' requires the ipykernel package.", you show run the code in the conda environment
```
pip install jupyterlab
```

# Starter walk-cliffing environment with mesa
With the well-desined api of mesa, we are able to make the agent and the model with the python framework to simulations.
The description of the walk-califfing environments is there: https://zhuanlan.zhihu.com/p/554765525
The basic elements there contianes:
1. Start agent, step actions is to choice the best ways, with the informations searching

2. hole, where should be not get

3. goal, reach the goal will let you get the maximum reward


In [78]:
import mesa
import numpy as np
# to build the model,four classes could be used: mesa.agent\mesa.model\mesa.space\mesa.time
# to anaylize the model, mesa.datacollection and mesa.batchrunner could be used
# to visualize the model, mesa.visualization.ModularVisualization and mesa.visualization.modules could be used
class MyAgent(mesa.Agent):
    def __init__(self, name, model):
        super().__init__(name, model)
        self.name = name
        self.type = 'agent'

    def step(self):
        print("{} activated".format(self.name))
        # Whatever else the agent does when activated
        # Random walk, only to nerest neighbors
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        # 不允许穿越边界
        possible_steps = [x for x in possible_steps if x[0]-self.pos[0] in [-1, 0, 1] and x[1]-self.pos[1] in [-1, 0, 1]]
        # 不允许走到 hole
        possible_steps = [x for x in possible_steps if self.model.grid.is_cell_empty(x) or x == (9,9)]
        # update the possible steps with the reward to goal
        possible_steps = self.reward_sort(possible_steps)
        # new_position = self.random.choice(possible_steps)
        new_position = possible_steps[0]
        self.model.grid.move_agent(self, new_position)
        if new_position == (9,9):
            self.model.schedule.agents.remove(self)
            print("Agent {} reached the goal".format(self.name))
            self.model.running = False
        
    def reward_sort(self,pos_list):
        # sort the pos_list by the reward to the goal
        # the reward is the manhattan distance to the goal
        goal = (9,9)
        pos_list = sorted(pos_list, key=lambda x: abs(x[0]-goal[0])+abs(x[1]-goal[1]))
        return pos_list

class Hole(mesa.Agent):
    def __init__(self, name, model):
        super().__init__(name, model)
        self.name = name
        self.type = 'hole'
        self.pos = (5, 5)
    
    def step(self):
        print("Hole activated", self.pos)

class Goal(mesa.Agent):
    def __init__(self, name, model):
        super().__init__(name, model)
        self.type = 'goal'
        self.name = name
    
    def step(self):
        print("Goal activated")


class MyModel(mesa.Model):
    def __init__(self, n_agents,n_holes):
        super().__init__()
        self.schedule = mesa.time.RandomActivation(self) # RandomActivation is a scheduler
        self.grid = mesa.space.MultiGrid(10, 10, torus=True) # MultiGrid is a grid
        for i in range(n_agents):
            a = MyAgent(i, self) # create an agent
            self.schedule.add(a) # add the agent to the scheduler

            goal = Goal('goal', self)
            self.schedule.add(goal)
            coords = (0,0)
            self.grid.place_agent(a, coords) # place the agent on the grid
            self.grid.place_agent(goal, (9,9))

        for i in range(n_holes):
            hole = Hole(i, self)
            self.schedule.add(hole)
            pos = (np.random.randint(2,9), np.random.randint(2,9))
            self.grid.place_agent(hole, pos)

        self.dc = mesa.datacollection.DataCollector(
            model_reporters = {'agent_nums': lambda m: m.schedule.get_agent_count()},
            agent_reporters = {'x': lambda a: a.pos[0], 'y': lambda a: a.pos[1]}
        )
    
    def step(self):
        self.schedule.step() # activate all agents
        self.dc.collect(self)

In [79]:
model = MyModel(1,5)
model.step()

Hole activated (8, 3)
Hole activated (2, 7)
0 activated
Hole activated (7, 2)
Hole activated (7, 6)
Hole activated (4, 3)
Goal activated


In [80]:
# 打印 agent 类型
for agent in model.schedule.agents:
    print(agent.type)

hole
hole
agent
hole
hole
hole
goal


In [82]:
# visual the agent based model 
def create_faces(agent):
    print(agent.type)
    portrayal = {"Shape": "circle", "r": 0.5, "Filled": "true", "Layer": 0}
    if agent.type == 'agent':
        portrayal["Color"] = "red"
    elif agent.type == 'hole':
        portrayal["Color"] = "black"
    elif agent.type == 'goal':
        portrayal["Color"] = "green"
    return portrayal

grid = mesa.visualization.CanvasGrid(create_faces, 10, 10, 500, 500)
server = mesa.visualization.ModularServer(MyModel, [grid], "My Model", {"n_agents": 1, "n_holes": 5})
server.launch(port=8528)

Interface starting at http://127.0.0.1:8528


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
agent
hole
hole
hole
hole
hole
goal


{"type":"get_step","step":1}
0 activated
Goal activated
Hole activated (3, 6)
Hole activated (6, 6)
Hole activated (3, 3)
Hole activated (6, 3)
Hole activated (2, 2)
agent
hole
hole
hole
hole
hole
goal
{"type":"get_step","step":2}
Hole activated (3, 3)
Hole activated (3, 6)
Hole activated (6, 6)
0 activated
Hole activated (2, 2)
Hole activated (6, 3)
Goal activated
agent
hole
hole
hole
hole
hole
goal
{"type":"get_step","step":3}
Hole activated (6, 6)
Hole activated (3, 3)
Goal activated
Hole activated (2, 2)
Hole activated (3, 6)
0 activated
Hole activated (6, 3)
hole
agent
hole
hole
hole
hole
goal
{"type":"get_step","step":4}
Hole activated (3, 3)
0 activated
Goal activated
Hole activated (6, 6)
Hole activated (6, 3)
Hole activated (3, 6)
Hole activated (2, 2)
hole
hole
agent
hole
hole
hole
goal
{"type":"get_step","step":5}
Hole activated (6, 6)
Hole activated (3, 3)
0 activated
Hole activated (2, 2)
Goal activated
Hole activated (6, 3)
Hole activated (3, 6)
hole
hole
hole
agent
hole
